In [1]:
%matplotlib inline

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import menpo.io as mio
from menpo.math import log_gabor
from menpo.feature import fast_dsift
from menpo.image import Image
from menpo.visualize import print_dynamic, progress_bar_str, visualize_images
from menpofit.visualize import visualize_fitting_results
from alaborticcv2015.alignment import (
    LKFitter, 
    FilteredSSD, FilteredFourierSSD, ECC, GradientImages, GradientCorrelation)
from alaborticcv2015.alignment.result import SerializableResult

# Experiment 2: YaleB Database, Previous Methods

In [2]:
db = mio.import_pickle('/Users/joan/Desktop/alaborticcv2015/data/yaleb.pkl.gz')

In [3]:
folder_path = '/Users/joan/Desktop/alaborticcv2015/results/yaleb/'

diagonal = 100
scales = (1,)
max_iters = 100

n_rep = 20
noise_std = range(2, 20, 2)

## SSD 

In [ ]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredSSD)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_ssd.pkl.gz', overwrite=True)

## LogGabor 

In [ ]:
def kernel_func(shape):
    k = np.fft.ifftshift(log_gabor(np.ones(shape[-2:]))[2]) 
    return k

errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredFourierSSD,
                          kernel_func=kernel_func)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_loggabor.pkl.gz', overwrite=True)

## ECC 

In [ ]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=ECC)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_ecc.pkl.gz', overwrite=True)

## Gradient Images 

In [ ]:
errors = np.empty((len(noise_std), len(yaleb), len(yaleb[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(yaleb)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = yaleb[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=GradientImages)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + '_gi.pkl.gz', overwrite=True)

## Gradient Correlation

In [ ]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=GradientCorrelation)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'yaleb_gc.pkl.gz', overwrite=True)

## dSift 

In [ ]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          features=fast_dsift,
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredSSD)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string1 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'yaleb_dsift.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  mean: 0.0152 - 0.0332  median: 0.0148 - 0.0293  std: 0.0045 - 0.0231 
- noise: 2,  sub: 1,  mean: 0.0152 - 0.0431  median: 0.0148 - 0.0347  std: 0.0045 - 0.0234 
- noise: 2,  sub: 2,  mean: 0.0152 - 0.0296  median: 0.0148 - 0.0239  std: 0.0045 - 0.0216 
- noise: 2,  sub: 3,  mean: 0.0152 - 0.0373  median: 0.0148 - 0.0343  std: 0.0045 - 0.0139 
- noise: 2,  sub: 4,  mean: 0.0152 - 0.0397  median: 0.0148 - 0.0359  std: 0.0045 - 0.0162 
- noise: 2,  sub: 5,  mean: 0.0152 - 0.0312  median: 0.0148 - 0.0218  std: 0.0045 - 0.0265 
- noise: 2,  sub: 6,  mean: 0.0152 - 0.0274  median: 0.0148 - 0.0246  std: 0.0045 - 0.0085 
- noise: 2,  sub: 7,  mean: 0.0152 - 0.0250  median: 0.0148 - 0.0245  std: 0.0045 - 0.0101 
- noise: 2,  sub: 8,  mean: 0.0152 - 0.0144  median: 0.0148 - 0.0126  std: 0.0045 - 0.0074 
- noise: 2,  sub: 9,  mean: 0.0152 - 0.0419  median: 0.0148 - 0.0384  std: 0.0045 - 0.0245 
- noise: 4,  sub: 0,  mean: 0.0304 - 0.0331  median: 0.0296 - 0.0302  std: 0.009

In [ ]:
def mean_error_per_noise_iter(errors):
    return np.mean(errors, axis=(0, 1, 2))

def plot_mean_error_per_noise_iter(errors):
    errors = mean_error_per_noise_iter(errors)
    
    plt.figure()
    plt.title('Normalized error per iteration')
    plt.ylabel('Normalized error')
    plt.xlabel('Iteration')
    plt.plot(range(errors.shape[0]), errors)

def convergence_per_noise(errors, threshold=0.03):
    final_errors = errors[..., -1]
    convergence_errors = np.empty(final_errors.shape[0])
    for j, errors in enumerate(final_errors):
        n_errors = np.prod(errors.shape)
        convergence_errors[j] = np.count_nonzero([errors <= threshold]) / n_errors
    return convergence_errors

def plot_convergence_per_noise(errors, threshold=0.03):
    errors = convergence_per_noise(errors, threshold=threshold)
    
    plt.figure()
    plt.ylabel('% converged')
    plt.xlabel('Noise sigma')
    plt.plot(range(errors.shape[0]), errors)